In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import dvu
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import neuro.features.feature_utils
import joblib
import neuro.data.story_names
import neuro.data.response_utils
from tqdm import tqdm
import neuro.features.feature_spaces
from himalaya.ridge import RidgeCV
from utils import generate_leave_one_run_out
from sklearn.model_selection import check_cv
from himalaya.backend import set_backend
import neuro.config
from tabpfn import TabPFNRegressor
from collections import defaultdict

In [ ]:
X = np.arange(100).reshape(-1, 1)

In [ ]:
def get_time_cols(n):

    t = np.arange(n) / n
    vals = [t]
    for period in np.logspace(0, 3, 5):
        vals.append(np.sin(t * period * 2 * np.pi))
        vals.append(np.cos(t * period * 2 * np.pi))
    return np.stack(vals, axis=1)


time_cols = get_time_cols(
    feats_train_subsampled.shape[0] + feats_test_subsampled.shape[0])
feats_train_subsampled_with_time = np.concatenate(
    [feats_train_subsampled, time_cols[:feats_train_subsampled.shape[0]]], axis=1)
feats_test_subsampled_with_time = np.concatenate(
    [feats_test_subsampled, time_cols[feats_train_subsampled.shape[0]:]], axis=1)

In [ ]:
for i in tqdm(range(preds_train.shape[1])):
    model = TabPFNRegressor(device='cuda:1')
    model.fit(feats_train_subsampled, preds_train[:, i])
    preds_test = model.predict(feats_test_subsampled)
    r['corrs_test_tabpfn'].append(np.corrcoef(
        preds_test, resps_test_subsampled_voxels[:, i])[0, 1])

    # redo with time feats
    model = TabPFNRegressor(device='cuda:1')
    model.fit(feats_train_subsampled_with_time, preds_train[:, i])
    preds_test = model.predict(feats_test_subsampled_with_time)
    r['corrs_test_tabpfn_time'].append(np.corrcoef(
        preds_test, resps_test_subsampled_voxels[:, i])[0, 1])

    print(
        f'voxel {i} test {r["corrs_test_ridge"][i]:.3f} -> {r["corrs_test_tabpfn"][-1]:.3f} -> {r["corrs_test_tabpfn_time"][-1]:.3f}')
    print(
        '\tavg cum. improvement',
        np.mean(r['corrs_test_tabpfn'] -
                np.mean(r['corrs_test_ridge'][:len(r['corrs_test_tabpfn'])])),
        np.mean(r['corrs_test_tabpfn_time'] -
                np.mean(r['corrs_test_ridge'][:len(r['corrs_test_tabpfn_time'])]))
    )